v2 made progress with not crashing the kernel but it's not fully compatible with the submission guidelines yet.

plus the MAE bottomed out at ~6.3, which isn't even enough to crack the first page.

In [1]:
import gc
import typing
import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext cudf.pandas
import pandas as pd
# import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [3]:
def regression_imputer(data:pd.DataFrame, target:str, drops:list=[]) -> pd.DataFrame:
    X_train = data[data[target].notna()].drop(drops, axis=1)
    y_train = data[target].loc[data.index.isin(X_train.index)]
    model = LinearRegression()
    model.fit(X_train, y_train)
    X_test = data[data[target].isna()].drop(drops, axis=1).dropna()
    y_test = model.predict(X_test)
    return pd.DataFrame(y_test, columns=[target], index=X_test.index)

In [4]:
def preprocess_unit(data:pd.DataFrame, imputes:list[str]=[], skips:list[str]=[], lags:int=0) -> pd.DataFrame:
    temp = data.copy()
    # impute
    for col in imputes:
        # imp = regression_imputer(temp, col, drops=imputes)
        temp = temp[col].fillna(0) #imp)
    temp = temp.interpolate()
    # normalize
    skipped = temp[skips]
    temp = temp.drop(skips, axis=1)
    temp = (temp - temp.min()) / (temp.max() - temp.min())
    # lag features
    lagged = []
    for i in range(1, lags+1):
        lag = temp.shift(i)
        lag.columns = [f'{col}_lag_{i}0s' for col in lag.columns]
        lagged.append(lag)
    # recombine
    temp = temp.join(lagged, how='inner').dropna(axis=0)
    temp = skipped.join(temp, how='inner')
    return temp

In [13]:
def preprocess(data:pd.DataFrame, **kwargs) -> pd.DataFrame:
    data = data.reset_index().set_index(['stock_id', 'date_id', 'seconds_in_bucket']).sort_index()
    preprocessed = []
    for stock_id in range(200):
        print(f'Preprocessing...{stock_id/2:04.1f}%', end='\r')
        stock = pd.DataFrame(data.loc[stock_id])
        stock = preprocess_unit(stock, **kwargs)
        stock['stock_id'] = stock_id
        preprocessed.append(stock)
    data = pd.concat(preprocessed).reset_index()
    print('Preprocessing...Complete')
    del preprocessed
    data = data.sort_values(by=['time_id'])
    data = data.drop(['stock_id', 'date_id', 'seconds_in_bucket', 'time_id'], axis=1)
    data = data.set_index('row_id')
    return data

In [6]:
data = pd.read_csv('./.data/train.csv', index_col='row_id')
data = data.dropna(subset=['target'])

In [7]:
y = data.target
X = data.drop('target', axis=1)
del data # memory is at a premium here

In [8]:
X = preprocess(X, imputes=['far_price', 'near_price'], skips=['time_id', 'row_id', 'imbalance_buy_sell_flag'], lags=6)

Preprocessing...Complete


In [9]:
class Model(typing.Protocol):
    def fit(self, X, y, sample_weight=None): ...
    def predict(self, X): ...

def train_model(model:Model, X:pd.DataFrame, y:pd.Series, folds:int=5) -> Model:
    for i, (i_train, i_valid) in enumerate(TimeSeriesSplit(n_splits=folds).split(X)):
        print(f'CV Fold {i + 1} of {folds} - Running...', end='\r')
        try:
            X_train, X_valid = X.iloc[i_train, :], X.iloc[i_valid, :]
            y_train, y_valid = y[i_train], y[i_valid]
            model.fit(X_train, y_train)
            y_pred = model.predict(X_valid)
            mae = mean_absolute_error(y_valid, y_pred)
            del X_train, X_valid, y_train, y_valid, y_pred # yep...
        except Exception as e:
            print(f'CV Fold {i + 1} of {folds} - Failed: {e}')
            print(f'Returning undertrained model ({i} folds)')
            break
        finally:
            gc.collect() # it's a LOT of data
        print(f'CV Fold {i + 1} of {folds} - Complete. MAE: {mae}')
    return model

In [10]:
model = train_model(XGBRegressor(seed=25), X, y, folds=4)

CV Fold 1 of 4 - Complete. MAE: 5.914791973668544
CV Fold 2 of 4 - Complete. MAE: 5.158742088106489
CV Fold 3 of 4 - Complete. MAE: 4.769280972739363
CV Fold 4 of 4 - Complete. MAE: 4.330236484052406


In [1]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [2]:
counter = 0
for (test, revealed_targets, sample_submission) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_submission.head(3))
    # X = preprocess(test, skips=['far_price', 'near_price', 'time_id', 'row_id', 'imbalance_buy_sell_flag'], lags=6)
    # sample_submission['target'] = model.predict(X)
    env.predict(sample_submission)
    counter += 1

ValueError: not enough values to unpack (expected 3, got 2)

Amazing progress but ultimately failed because I can't get submissions to work. I need to re-design my process around it for v4.